In [1]:
import datetime as dt
import pandas as pd
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [4]:
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")

In [5]:
df = df_.copy()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [8]:
df.shape

(541910, 8)

In [10]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom


 2010-2011 UK müşterileri için 6 aylık CLTV prediction yapınız.

In [9]:
df = df[df["Country"] == "United Kingdom"]
df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Quantity"] > 0]


Veri setindeki aykırı değerleri gidermek için eşik değer belirlenir.

In [12]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

Aykırı değerler belirlenen eşik değerler ile değiştirilir.

In [13]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


In [15]:
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

TotalPrice adında yeni değişken tanımlıyoruz.

In [16]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [17]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.00000,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom,15.30000
1,536365,71053,WHITE METAL LANTERN,6.00000,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.00000,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom,22.00000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.00000,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.00000,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000


Analizde kullanacağımız zaman:

In [18]:
today_date = dt.datetime(2011, 12, 11)

## Metriklerin Hazırlanması

Recency: Son alışveriş ve ilk alışveriş arasındaki fark

Tenure: İlk alışveriş tarihinden bugüne kadar olan zaman. Müşteri yaşı.

Frequency: Tekrar eden alışveriş sayısı. 1'den büyük olanları alıyoruz.

Monetary: Satın alma başına ortalama kazanç.

In [19]:
cltv_df = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                                                         lambda date: (today_date - date.min()).days],
                                         'Invoice': lambda num: num.nunique(),
                                         'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

In [20]:
cltv_df.columns = cltv_df.columns.droplevel(0)

In [21]:
cltv_df.head()

,<lambda_0>,<lambda_1>,<lambda>,<lambda>
Customer ID,,,,
12346.00000,0,326,1,258.44000
12747.00000,366,370,11,4196.01000
12748.00000,372,374,210,32303.91000
12749.00000,209,214,5,4072.44000
12820.00000,323,327,4,942.34000


Değişken adlarını metrik adlarıyla değiştiriyoruz.

In [22]:
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']

In [23]:
cltv_df.head()

,recency,T,frequency,monetary
Customer ID,,,,
12346.00000,0,326,1,258.44000
12747.00000,366,370,11,4196.01000
12748.00000,372,374,210,32303.91000
12749.00000,209,214,5,4072.44000
12820.00000,323,327,4,942.34000


Monetary değeri burada RFM'deki gibi toplam olarak değil ortalama olarak hesaplanmaktadır. Ve sıfırdan büyük değerler dahil edilir.

In [24]:
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]

Haftalık cinsten ifade edilme.

In [25]:
cltv_df["recency"] = cltv_df["recency"] / 7
cltv_df["T"] = cltv_df["T"] / 7

Frequency değerinde 1'den büyük değerleri alıyoruz. Çünkü 1 kere alışveriş yapan müşteri kısmi churn olmuştur.

In [26]:
cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

In [27]:
cltv_df.head()

,recency,T,frequency,monetary
Customer ID,,,,
12747.00000,52.28571,52.85714,11,381.45545
12748.00000,53.14286,53.42857,210,153.82814
12749.00000,29.85714,30.57143,5,814.48800
12820.00000,46.14286,46.71429,4,235.58500
12822.00000,2.28571,12.57143,2,474.44000


BG - NBD MODELİ

In [28]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'])

<lifetimes.BetaGeoFitter: fitted with 2570 subjects, a: 0.12, alpha: 11.66, b: 2.51, r: 2.21>

GAMMA GAMMA MODELİ

In [30]:
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                        cltv_df['monetary']).sort_values(ascending=False).head(10)

Customer ID
14088.00000   3911.31885
18102.00000   3595.19256
14096.00000   3191.38673
17511.00000   2938.27449
15749.00000   2675.11900
17450.00000   2639.41934
13081.00000   2616.62439
16984.00000   2452.12491
16000.00000   2181.32360
16684.00000   2133.20359
dtype: float64

In [31]:
cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                             cltv_df['monetary'])

In [32]:
cltv_df.sort_values("expected_average_profit", ascending=False).head(10)

,recency,T,frequency,monetary,expected_average_profit
Customer ID,,,,,
14088.00000,44.57143,46.14286,13,3859.60154,3911.31885
18102.00000,52.28571,52.57143,60,3584.88775,3595.19256
14096.00000,13.85714,14.57143,17,3159.07706,3191.38673
17511.00000,52.85714,53.42857,31,2921.95194,2938.27449
15749.00000,13.85714,47.57143,3,2521.44667,2675.11900
17450.00000,51.28571,52.57143,46,2629.52989,2639.41934
13081.00000,51.28571,53.14286,11,2575.62273,2616.62439
16984.00000,5.85714,18.71429,2,2240.67500,2452.12491
16000.00000,0.00000,0.42857,3,2055.78667,2181.32360


6 AYLIK TAHMİN

In [33]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=6, 
                                   freq="W",  
                                   discount_rate=0.01)

In [34]:
cltv.head()

Customer ID
12747.00000    1937.04614
12748.00000   12365.79618
12749.00000    3446.01044
12820.00000     631.93933
12822.00000    1612.09665
Name: clv, dtype: float64

In [35]:
cltv.shape

(2570,)

In [36]:
cltv = cltv.reset_index()

In [37]:
cltv.head()

,Customer ID,clv
0,12747.00000,1937.04614
1,12748.00000,12365.79618
2,12749.00000,3446.01044
3,12820.00000,631.93933
4,12822.00000,1612.09665


In [38]:
cltv.sort_values(by="clv", ascending=False).head(10)

,Customer ID,clv
2486,18102.00000,85651.01047
589,14096.00000,55650.64677
2184,17450.00000,48533.31011
2213,17511.00000,36797.00673
1804,16684.00000,25083.02541
406,13694.00000,25060.70871
587,14088.00000,25010.05913
1173,15311.00000,23591.38948
133,13089.00000,22927.69296
1057,15061.00000,21123.08206


In [66]:
cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")
cltv_final.sort_values(by="clv", ascending=False).head(10)

,recency,T,frequency,monetary,expected_average_profit,clv
Customer ID,,,,,,
18102.00000,52.28571,52.57143,60,3584.88775,3595.19256,85651.01047
14096.00000,13.85714,14.57143,17,3159.07706,3191.38673,55650.64677
17450.00000,51.28571,52.57143,46,2629.52989,2639.41934,48533.31011
17511.00000,52.85714,53.42857,31,2921.95194,2938.27449,36797.00673
16684.00000,50.42857,51.28571,28,2120.04696,2133.20359,25083.02541
13694.00000,52.71429,53.42857,50,1267.36260,1271.78540,25060.70871
14088.00000,44.57143,46.14286,13,3859.60154,3911.31885,25010.05913
15311.00000,53.28571,53.42857,91,667.59681,668.89446,23591.38948
13089.00000,52.28571,52.85714,97,605.18660,606.29366,22927.69296


İlk müşteri için yorum yapılacak olursa recency ve tenure değeri çok yakın. Frequency değeri yüksek ve monetary yüksek böylelikle clv değeri yüksek. 
İlk ve ikinci müşteriye birlikte bakılırsa ikinci müşteri yeni bir kullanıcıdır ve frequency değeri ilk müşterden oldukça düşük ancak monetary değerleri yakındır.
İki ve üçüncü müşterilerin frequency ve monetary değerlerine bakılırsa monetary değeri baskın görünüyor. Aynı kazancı sağladıktan sonra işlem sayısı etkisini kaybediyor.
Genelleştirecek olursak, recency ve tenure değerlerinin yakın olması clv açısından pozitif bir etken. Frequency ve monetary değerleri incelenecek olursa da düşük frequency değerine sahip müşterileri yukarıya taşıyan etken monetary değeridir. Az ama öz işlem yapılmış denebilir.

In [41]:
cltv1 = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=1, 
                                   freq="W",  
                                   discount_rate=0.01)

In [43]:
rfm_cltv1_final = cltv_df.merge(cltv1, on="Customer ID", how="left")
rfm_cltv1_final.sort_values(by="clv", ascending=False).head()

,recency,T,frequency,monetary,expected_average_profit,clv
Customer ID,,,,,,
18102.00000,52.28571,52.57143,60,3584.88775,3595.19256,14884.97500
14096.00000,13.85714,14.57143,17,3159.07706,3191.38673,9855.87989
17450.00000,51.28571,52.57143,46,2629.52989,2639.41934,8434.76472
17511.00000,52.85714,53.42857,31,2921.95194,2938.27449,6394.32432
16684.00000,50.42857,51.28571,28,2120.04696,2133.20359,4361.05396


In [45]:
cltv12 = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=12, 
                                   freq="W",  
                                   discount_rate=0.01)

In [46]:
rfm_cltv12_final = cltv_df.merge(cltv12, on="Customer ID", how="left")
rfm_cltv12_final.sort_values(by="clv", ascending=False).head()

,recency,T,frequency,monetary,expected_average_profit,clv
Customer ID,,,,,,
18102.00000,52.28571,52.57143,60,3584.88775,3595.19256,163591.12676
14096.00000,13.85714,14.57143,17,3159.07706,3191.38673,104900.44292
17450.00000,51.28571,52.57143,46,2629.52989,2639.41934,92694.27482
17511.00000,52.85714,53.42857,31,2921.95194,2938.27449,70285.65230
16684.00000,50.42857,51.28571,28,2120.04696,2133.20359,47890.36421


In [47]:
rfm_cltv1_final.sort_values("clv", ascending=False).head(10)

,recency,T,frequency,monetary,expected_average_profit,clv
Customer ID,,,,,,
18102.00000,52.28571,52.57143,60,3584.88775,3595.19256,14884.97500
14096.00000,13.85714,14.57143,17,3159.07706,3191.38673,9855.87989
17450.00000,51.28571,52.57143,46,2629.52989,2639.41934,8434.76472
17511.00000,52.85714,53.42857,31,2921.95194,2938.27449,6394.32432
16684.00000,50.42857,51.28571,28,2120.04696,2133.20359,4361.05396
14088.00000,44.57143,46.14286,13,3859.60154,3911.31885,4355.48526
13694.00000,52.71429,53.42857,50,1267.36260,1271.78540,4354.46853
15311.00000,53.28571,53.42857,91,667.59681,668.89446,4098.86945
13089.00000,52.28571,52.85714,97,605.18660,606.29366,3984.05150


In [48]:
rfm_cltv12_final.sort_values("clv", ascending=False).head(10)

,recency,T,frequency,monetary,expected_average_profit,clv
Customer ID,,,,,,
18102.00000,52.28571,52.57143,60,3584.88775,3595.19256,163591.12676
14096.00000,13.85714,14.57143,17,3159.07706,3191.38673,104900.44292
17450.00000,51.28571,52.57143,46,2629.52989,2639.41934,92694.27482
17511.00000,52.85714,53.42857,31,2921.95194,2938.27449,70285.65230
16684.00000,50.42857,51.28571,28,2120.04696,2133.20359,47890.36421
13694.00000,52.71429,53.42857,50,1267.36260,1271.78540,47871.90000
14088.00000,44.57143,46.14286,13,3859.60154,3911.31885,47688.86365
15311.00000,53.28571,53.42857,91,667.59681,668.89446,45067.80939
13089.00000,52.28571,52.85714,97,605.18660,606.29366,43795.47286


In [49]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=6, 
                                   freq="W",  
                                   discount_rate=0.01)

In [50]:
cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")
cltv_final.head()

,recency,T,frequency,monetary,expected_average_profit,clv
Customer ID,,,,,,
12747.00000,52.28571,52.85714,11,381.45545,387.82285,1937.04614
12748.00000,53.14286,53.42857,210,153.82814,153.97132,12365.79618
12749.00000,29.85714,30.57143,5,814.48800,844.09470,3446.01044
12820.00000,46.14286,46.71429,4,235.58500,247.08095,631.93933
12822.00000,2.28571,12.57143,2,474.44000,520.82819,1612.09665


In [51]:
cltv_final["cltv_segment"] = pd.qcut(cltv_final["clv"], 4, labels=["D", "C", "B", "A"])
cltv_final.head()

,recency,T,frequency,monetary,expected_average_profit,clv,cltv_segment
Customer ID,,,,,,,
12747.00000,52.28571,52.85714,11,381.45545,387.82285,1937.04614,A
12748.00000,53.14286,53.42857,210,153.82814,153.97132,12365.79618,A
12749.00000,29.85714,30.57143,5,814.48800,844.09470,3446.01044,A
12820.00000,46.14286,46.71429,4,235.58500,247.08095,631.93933,C
12822.00000,2.28571,12.57143,2,474.44000,520.82819,1612.09665,B


In [61]:
cltv_final.groupby("cltv_segment").agg({"mean"})

,recency,T,frequency,monetary,expected_average_profit,clv
,mean,mean,mean,mean,mean,mean
cltv_segment,,,,,,
D,22.06710,40.50900,3.06843,177.42499,192.22096,269.46705
C,30.83645,38.16555,3.99065,260.68927,278.06975,710.82370
B,29.83890,35.11704,5.45483,351.98686,370.49558,1271.35290
A,31.46056,34.51922,11.29238,586.30221,608.69662,3806.34897


CLTV değerleri yüksek olan A ve B segmentlerinde recency ve tenure değerleri arasında fark azdır. Bu durum pozitif bir etkiye sahiptir. A segmentinde ayrıca frekans ve monetary ortalaması değeri oldukça yüksektir. Diğer segmentlere göre A segmentinin Tenure değeri daha düşüktür yani daha yeni müşterilere sahiptir. Bu segmentteki müşterilere odaklanarak kazanç yükseltilebilir. D segmentindeki müşterilere de çeşitli kampanyalarla alışveriş hatırlatmaları yapılabilir. 